In [267]:
#Import Packages 
import pandas as pd
import requests
from datetime import datetime

In [5]:
#Connection with Mysql
schema = "cities_gans"
host = "127.0.0.1"
user = "root"
password = "Focasql9"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [7]:
#Get cities information from SQL
cities_info = pd.read_sql('cities_data', con=connection_string)
cities_info

,City_id,City,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.40500
1,2,Hamburg,Germany,53.5500,10.00000
2,3,Munich,Germany,48.1375,11.57500
3,4,Cologne,Germany,50.9364,6.95278
4,5,Paris,France,48.8567,2.35222
5,6,London,United Kingdom,51.5072,-0.12750
6,7,Madrid,Spain,40.4169,-3.70333
7,8,Copenhagen,Denmark,55.6761,12.56830
8,9,Vienna,Austria,48.2083,16.37250
9,10,Budapest,Hungary,47.4925,19.05140


In [242]:
cities_info['City'].tolist()

['Berlin',
 'Hamburg',
 'Munich',
 'Cologne',
 'Paris',
 'London',
 'Madrid',
 'Copenhagen',
 'Vienna',
 'Budapest',
 'Prague']

In [146]:
lat_float = cities_info.loc[cities_info['City']== 'Berlin','Latitude'].iloc[0]
lon_float = cities_info.loc[cities_info['City']== 'Berlin','Longitude'].iloc[0]

In [147]:
lat = f"{lat_float:.4f}"
lon = f"{lon_float:.5f}"
lon

'13.40500'

In [285]:
# Define the parameters to be passed into the url.
#Ej: latitude and longitude for Berlin
lat = f"{cities_info.loc[cities_info['City']== 'Prague','Latitude'].iloc[0]:.4f}"
lon = f"{cities_info.loc[cities_info['City']== 'Prague','Longitude'].iloc[0]:.5f}"
API_key = "d2c1e0da49b926a614a4f66b4c1fd01b"

In [33]:
#API Call ej
f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric"

'https://api.openweathermap.org/data/2.5/forecast?lat=52.5200&lon=13.40500&appid=d2c1e0da49b926a614a4f66b4c1fd01b&units=metric'

In [287]:
lon

'14.42140'

In [289]:
weather_request = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric")
weather_json = weather_request.json()
weather_json

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [152]:
#Get the keys of the json file
weather_json.keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [362]:
#Weather information in the key 'list' - Weather, main
weather_json['list'][0]['main']['temp'] #Get temp

weather_json['list'][0]['main']['feels_like'] #Get feels like [0] is the item, the forecast_date

weather_json['list'][0]['wind']['speed'] #Get wind speed

weather_json['list'][0]['pop'] #Get rain probability

weather_json['list'][10]['rain'].get('3h',0) #Get rain - if there is no rain, put 0

weather_json['list'][0]['main']['temp_min'] # Get min temperature

weather_json['list'][0]['main']['temp_max']

20.46

In [154]:
#Get date time
dt = weather_json['list'][0]['dt']

#Get main - temp, feel, etc
weather_json['list'][0]['main']

#Get main - temp, feel, etc
main_w = weather_json['list'][0]['weather'][0]['main']
weath_desc = weather_json['list'][0]['weather'][0]['description']

dt_txt = weather_json['list'][0]['dt_txt']

dt, main_w,  weath_desc, dt_txt

(1747666800, 'Clouds', 'overcast clouds', '2025-05-19 15:00:00')

In [330]:
weather_list = weather_json.get('list', [])
rain = []
#weather_json['list'][0]['wind']['speed']
#weather_json['list'][0]#['rain']#.get('3h',0)

for item in weather_list:
    if 'rain' in item:
        rain.append(item['rain'].get('3h'))
    else:
        rain.append(0)

len(rain) 

40

In [323]:
dates = []
weather_list = weather_json.get('list', [])  # Obtiene la lista de forma segura

for item in weather_list:
    if 'dt_txt' in item:
        dates.append(item['dt_txt'])

len(dates)

40

In [101]:
#Get everything in one loop
main_weather = []
description_weather = []
dates = []

weather_list = weather_json.get('list', [])

for item in weather_list:
    
    if 'weather' in item:
        #For each date get the main weather and append it to the main_weather list
        main_weather.append(item['weather'][0]['main'])
        #For each date get the description of the weather and append it to the description_weather list
        description_weather.append(item['weather'][0]['description'])
    
    if 'dt_txt' in item:
        #For each date get the date and append it to the dates list
        dates.append(item['dt_txt'])


main_weather, description_weather, dates



(['Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clear',
  'Clear',
  'Clear',
  'Clear',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Rain',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clear',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clear',
  'Clouds',
  'Clouds',
  'Clear',
  'Clouds',
  'Clouds',
  'Clouds',
  'Rain',
  'Clouds',
  'Clouds',
  'Clouds',
  'Clouds',
  'Rain',
  'Clouds'],
 ['overcast clouds',
  'overcast clouds',
  'broken clouds',
  'scattered clouds',
  'clear sky',
  'clear sky',
  'clear sky',
  'clear sky',
  'broken clouds',
  'broken clouds',
  'overcast clouds',
  'broken clouds',
  'overcast clouds',
  'light rain',
  'overcast clouds',
  'overcast clouds',
  'overcast clouds',
  'broken clouds',
  'overcast clouds',
  'broken clouds',
  'broken clouds',
  'scattered clouds',
  'clear sky',
  'few clouds',
  'broken clouds',
  'broken clouds',
  'clear sky',
  'few clouds',
  'few clou

In [102]:
#Create a weather DataFrame for one city using list
weather_df = pd.DataFrame(list(zip(dates, main_weather, description_weather)),
                          columns=['Date', 'Main_weather', 'Description_weather'])
weather_df

,Date,Main_weather,Description_weather
0,2025-05-19 15:00:00,Clouds,overcast clouds
1,2025-05-19 18:00:00,Clouds,overcast clouds
2,2025-05-19 21:00:00,Clouds,broken clouds
3,2025-05-20 00:00:00,Clouds,scattered clouds
4,2025-05-20 03:00:00,Clear,clear sky
5,2025-05-20 06:00:00,Clear,clear sky
6,2025-05-20 09:00:00,Clear,clear sky
7,2025-05-20 12:00:00,Clear,clear sky
8,2025-05-20 15:00:00,Clouds,broken clouds
9,2025-05-20 18:00:00,Clouds,broken clouds


In [185]:
#Set everything in one function

# Input a list of cities, do this for every city, for city in cities...
# 1. Create one function to use the API and get the json file for a city

def json_city(city): 
    print(city)
    lat = f"{cities_info.loc[cities_info['City']== city,'Latitude'].iloc[0]:.4f}"
    print(lat)
    lon = f"{cities_info.loc[cities_info['City']== city,'Longitude'].iloc[0]:.4f}"
    print(lon)
    API_key = "d2c1e0da49b926a614a4f66b4c1fd01b"
    
    city_request = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric")
    city_json = city_request.json()
    return city_json

In [231]:
# 2. Create a function to get the lists of date, main weather, description weather

def get_weather_info(city_json):
    
    main_weather = []
    description_weather = []
    temperatures = []
    dates = []
    city_name = []

    weather_list = city_json.get('list', [])

    for item in weather_list: 
        
        if 'weather' in item:
            #For each date get the main weather and append it to the main_weather list
            main_weather.append(item['weather'][0]['main'])
            #For each date get the description of the weather and append it to the description_weather list
            description_weather.append(item['weather'][0]['description'])

        if 'dt_txt' in item:
            #For each date get the date and append it to the dates list
            dates.append(item['dt_txt'])
        
        if 'main' in item:
            #For each date append the temperature
            temperatures.append(item['main']['temp'])
            
        #Append city name in each row
        city_name.append(city_json['city']['name'])

    #Create a weather data frame for the city using the lists
    
    weather_df = pd.DataFrame(list(zip(city_name, dates, temperatures, main_weather, description_weather)),
                          columns=['City','Date', 'Temperature','Main_weather', 'Description_weather'])
            
    return weather_df

In [232]:
json_hamburg = json_city('Hamburg')

Hamburg
53.5500
10.0000


In [233]:
get_weather_info(json_hamburg)

,City,Date,Temperature,Main_weather,Description_weather
0,Hamburg,2025-05-19 18:00:00,19.57,Clouds,overcast clouds
1,Hamburg,2025-05-19 21:00:00,15.78,Clouds,overcast clouds
2,Hamburg,2025-05-20 00:00:00,10.31,Clouds,broken clouds
3,Hamburg,2025-05-20 03:00:00,8.88,Clear,clear sky
4,Hamburg,2025-05-20 06:00:00,13.95,Clear,clear sky
5,Hamburg,2025-05-20 09:00:00,19.86,Clear,clear sky
6,Hamburg,2025-05-20 12:00:00,22.04,Clear,clear sky
7,Hamburg,2025-05-20 15:00:00,20.53,Clouds,scattered clouds
8,Hamburg,2025-05-20 18:00:00,16.42,Clouds,scattered clouds
9,Hamburg,2025-05-20 21:00:00,9.81,Clouds,few clouds


In [363]:
# 3. Wrap everything in a for to iterate in each city

def weather_cities(cities_info): #Cambiarlo para que reciba el cities_info data frame
    
    #Create an empty list of the cities data frames
    cities_dataframes = []
    
    cities = cities_info['City'].tolist()
    
    for city in cities:
        
        #Get row of the city in the cities_info data frame
        city_row = cities_info[cities_info['City']==city].iloc[0]
        city_id = city_row['City_id']
        
        # 1. Create one function to use the API and get the json file for a city
        
        #Get the latitude of the city from the cities_info data frame
        lat = f"{city_row['Latitude']:.4f}"
    
        #Get the longitude of the city from the cities_info data frame
        lon = f"{city_row['Longitude']:.5f}"
        
        API_key = "d2c1e0da49b926a614a4f66b4c1fd01b"

        #Save respond of the opnen weather map website in the variable city_request
        city_request = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric")

        #Save the response as a json file 
        city_json = city_request.json()
    
        #2. Get the lists of date, main weather, description weather from the city_json file
    
        #Create empty lists to store the data response from de API
        main_weather = []
        description_weather = []
        temperatures = []
        dates = []
        feels_like = []
        min_temp = []
        max_temp = []
        wind_speed = []
        rain_probability = []
        rain = []
        city_name = []
        data_retrieved = []

        #Get the items of the 'list' in the json file
        weather_list = city_json.get('list', [])

        for item in weather_list: 

            if 'weather' in item:
                #For each date get the main weather and append it to the main_weather list
                main_weather.append(item['weather'][0]['main'])
                #For each date get the description of the weather and append it to the description_weather list
                description_weather.append(item['weather'][0]['description'])

            if 'dt_txt' in item:
                #For each date get the date and append it to the dates list
                dates.append(item['dt_txt'])
            
            if 'main' in item:
                #For each date append the temperature
                temperatures.append(item['main']['temp'])
                
                #For each date append the feels like temperature
                feels_like.append(item['main']['feels_like'])
                
                #For each Forecast moment append the min temperature ('temp_min')
                min_temp.append(item['main']['temp_min'])
                
                #For each Forecast moment append the max temperature ('temp_max')
                max_temp.append(item['main']['temp_max'])
            
            if 'wind' in item:
                #For each forecast moment append wind speed
                wind_speed.append(item['wind']['speed'])
            
            if 'pop' in item:
                #For each forecast moment append probability of precipitation - Rain probability
                rain_probability.append(item['pop'])
            
            if 'rain' in item:
                rain.append(item['rain'].get('3h'))
            else:
                rain.append(0)

            #Append city name in each row
            city_name.append(city)
            
            #Append today date to each row
            data_retrieved.append(datetime.today().strftime("%Y-%m-%d %H:%M:%S"))

        #Organice the data in one Data frame per city
        city_df = pd.DataFrame({'City': city_name,
                                'Forecast_time': dates,
                                'Temperature': temperatures,
                                'Feels_like': feels_like,
                                'Min_temp': min_temp,
                                'Max_temp': max_temp,
                                'Main_weather': main_weather,
                                'Description_weather': description_weather,
                                'Wind_speed': wind_speed,
                                'Rain_prob': rain_probability,
                                'Rain': rain,
                                'Data_retrieved': data_retrieved
                                })
        
        #Insert city_id to the dataframe in the first column position 
        city_df.insert(0, 'City_id', city_id)
        
        #Append city_df in the data frames list
        cities_dataframes.append(city_df)
        
    #Concatenate all data frames in one final df
    weather_df = pd.concat(cities_dataframes, ignore_index=True)
    
    #Change date columns to datetime format
    weather_df['Forecast_time'] = pd.to_datetime(weather_df['Forecast_time'])
    weather_df['Data_retrieved'] = pd.to_datetime(weather_df['Data_retrieved'])
    
    return weather_df


In [364]:
weather_df = weather_cities(cities_info)
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   City_id              440 non-null    int64         
 1   City                 440 non-null    object        
 2   Forecast_time        440 non-null    datetime64[ns]
 3   Temperature          440 non-null    float64       
 4   Feels_like           440 non-null    float64       
 5   Min_temp             440 non-null    float64       
 6   Max_temp             440 non-null    float64       
 7   Main_weather         440 non-null    object        
 8   Description_weather  440 non-null    object        
 9   Wind_speed           440 non-null    float64       
 10  Rain_prob            440 non-null    float64       
 11  Rain                 440 non-null    float64       
 12  Data_retrieved       440 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(7), i

In [367]:
weather_df

,City_id,City,Forecast_time,Temperature,Feels_like,Min_temp,Max_temp,Main_weather,Description_weather,Wind_speed,Rain_prob,Rain,Data_retrieved
0,1,Berlin,2025-05-20 15:00:00,23.27,22.80,22.54,23.27,Clouds,overcast clouds,2.81,0.00,0.00,2025-05-20 16:10:33
1,1,Berlin,2025-05-20 18:00:00,22.25,21.76,20.22,22.25,Clouds,overcast clouds,2.27,0.00,0.00,2025-05-20 16:10:33
2,1,Berlin,2025-05-20 21:00:00,20.05,19.63,18.44,20.05,Clouds,overcast clouds,2.33,0.00,0.00,2025-05-20 16:10:33
3,1,Berlin,2025-05-21 00:00:00,15.20,14.66,15.20,15.20,Clouds,overcast clouds,2.74,0.00,0.00,2025-05-20 16:10:33
4,1,Berlin,2025-05-21 03:00:00,11.40,10.29,11.40,11.40,Rain,light rain,3.02,0.54,0.37,2025-05-20 16:10:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,11,Prague,2025-05-25 00:00:00,10.43,9.12,10.43,10.43,Clouds,overcast clouds,2.88,0.00,0.00,2025-05-20 16:10:43
436,11,Prague,2025-05-25 03:00:00,11.76,10.43,11.76,11.76,Clouds,overcast clouds,2.78,0.00,0.00,2025-05-20 16:10:43
437,11,Prague,2025-05-25 06:00:00,11.99,10.97,11.99,11.99,Rain,light rain,4.87,0.20,0.10,2025-05-20 16:10:43
438,11,Prague,2025-05-25 09:00:00,11.84,11.14,11.84,11.84,Rain,light rain,4.83,0.67,0.36,2025-05-20 16:10:43


In [366]:
#Send the weather table to SQL
weather_df.to_sql(name='weather',
                  if_exists='append',
                  con=connection_string,
                  index=False) 

440

In [368]:
weather_in_db = pd.read_sql("SELECT * FROM weather", con=connection_string)
weather_in_db

,City_id,City,Forecast_time,Temperature,Feels_like,Min_temp,Max_temp,Main_weather,Description_weather,Wind_speed,Rain_prob,Rain,Data_retrieved
0,1,Berlin,2025-05-20 15:00:00,23.27,22.80,22.54,23.27,Clouds,overcast clouds,2.81,0.00,0.00,2025-05-20 16:10:33
1,1,Berlin,2025-05-20 18:00:00,22.25,21.76,20.22,22.25,Clouds,overcast clouds,2.27,0.00,0.00,2025-05-20 16:10:33
2,1,Berlin,2025-05-20 21:00:00,20.05,19.63,18.44,20.05,Clouds,overcast clouds,2.33,0.00,0.00,2025-05-20 16:10:33
3,1,Berlin,2025-05-21 00:00:00,15.20,14.66,15.20,15.20,Clouds,overcast clouds,2.74,0.00,0.00,2025-05-20 16:10:33
4,1,Berlin,2025-05-21 03:00:00,11.40,10.29,11.40,11.40,Rain,light rain,3.02,0.54,0.37,2025-05-20 16:10:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,11,Prague,2025-05-25 00:00:00,10.43,9.12,10.43,10.43,Clouds,overcast clouds,2.88,0.00,0.00,2025-05-20 16:10:43
436,11,Prague,2025-05-25 03:00:00,11.76,10.43,11.76,11.76,Clouds,overcast clouds,2.78,0.00,0.00,2025-05-20 16:10:43
437,11,Prague,2025-05-25 06:00:00,11.99,10.97,11.99,11.99,Rain,light rain,4.87,0.20,0.10,2025-05-20 16:10:43
438,11,Prague,2025-05-25 09:00:00,11.84,11.14,11.84,11.84,Rain,light rain,4.83,0.67,0.36,2025-05-20 16:10:43


## Final functions

In [372]:
# Create a function to create the connection string
def create_connection_string(): 
    schema = "cities_gans" #Name of the database in MySQL
    host = "127.0.0.1"     #Local host IP
    user = "root"          #Database username
    password = "Focasql9"
    port = 3306
    
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [370]:
# Read the 'cities_data' table into a DataFrame
def read_cities_data(connection_string):
    return pd.read_sql("cities_data", con=connection_string)

In [ ]:
#Function to call OpenWeatherMap API and return DataFrame with weather forecast
def weather_cities(cities_info): 
    
    #Create an empty list of the cities data frames
    cities_dataframes = []
    
    cities = cities_info['City'].tolist()
    
    for city in cities:
        
        #Get row of the city in the cities_info data frame
        city_row = cities_info[cities_info['City']==city].iloc[0]
        city_id = city_row['City_id']
        
        # 1. Create one function to use the API and get the json file for a city
        
        #Get the latitude of the city from the cities_info data frame
        lat = f"{city_row['Latitude']:.4f}"
    
        #Get the longitude of the city from the cities_info data frame
        lon = f"{city_row['Longitude']:.5f}"
        
        API_key = "d2c1e0da49b926a614a4f66b4c1fd01b"

        #Save respond of the opnen weather map website in the variable city_request
        city_request = requests.get(f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API_key}&units=metric")

        #Save the response as a json file 
        city_json = city_request.json()
    
        #2. Get the lists of date, main weather, description weather from the city_json file
    
        #Create empty lists to store the data response from de API
        main_weather = []
        description_weather = []
        temperatures = []
        dates = []
        feels_like = []
        min_temp = []
        max_temp = []
        wind_speed = []
        rain_probability = []
        rain = []
        city_name = []
        data_retrieved = []

        #Get the items of the 'list' in the json file
        weather_list = city_json.get('list', [])

        for item in weather_list: 

            if 'weather' in item:
                #For each date get the main weather and append it to the main_weather list
                main_weather.append(item['weather'][0]['main'])
                #For each date get the description of the weather and append it to the description_weather list
                description_weather.append(item['weather'][0]['description'])

            if 'dt_txt' in item:
                #For each date get the date and append it to the dates list
                dates.append(item['dt_txt'])
            
            if 'main' in item:
                #For each date append the temperature
                temperatures.append(item['main']['temp'])
                
                #For each date append the feels like temperature
                feels_like.append(item['main']['feels_like'])
                
                #For each Forecast moment append the min temperature ('temp_min')
                min_temp.append(item['main']['temp_min'])
                
                #For each Forecast moment append the max temperature ('temp_max')
                max_temp.append(item['main']['temp_max'])
            
            if 'wind' in item:
                #For each forecast moment append wind speed
                wind_speed.append(item['wind']['speed'])
            
            if 'pop' in item:
                #For each forecast moment append probability of precipitation - Rain probability
                rain_probability.append(item['pop'])
            
            if 'rain' in item:
                rain.append(item['rain'].get('3h'))
            else:
                rain.append(0)

            #Append city name in each row
            city_name.append(city)
            
            #Append today date to each row
            data_retrieved.append(datetime.today().strftime("%Y-%m-%d %H:%M:%S"))

        #Organice the data in one Data frame per city
        city_df = pd.DataFrame({'City': city_name,
                                'Forecast_time': dates,
                                'Temperature': temperatures,
                                'Feels_like': feels_like,
                                'Min_temp': min_temp,
                                'Max_temp': max_temp,
                                'Main_weather': main_weather,
                                'Description_weather': description_weather,
                                'Wind_speed': wind_speed,
                                'Rain_prob': rain_probability,
                                'Rain': rain,
                                'Data_retrieved': data_retrieved
                                })
        
        #Insert city_id to the dataframe in the first column position 
        city_df.insert(0, 'City_id', city_id)
        
        #Append city_df in the data frames list
        cities_dataframes.append(city_df)
        
    #Concatenate all data frames in one final df
    weather_df = pd.concat(cities_dataframes, ignore_index=True)
    
    #Change date columns to datetime format
    weather_df['Forecast_time'] = pd.to_datetime(weather_df['Forecast_time'])
    weather_df['Data_retrieved'] = pd.to_datetime(weather_df['Data_retrieved'])
    
    return weather_df

In [369]:
#Create a function to send the weather DataFrame in the SQL table named 'weather'
def save_weather_data(weather_df,connection_string):
    weather_df.to_sql(name = 'weather',
                      if_exists = 'append', #Append new rows
                      con = connection_string,
                      index = False)

In [373]:
#Create a function to retrieve and send data - whole workflow
def receive_and_send_data():
    #1. Create SQL connection string
    connection_string = create_connection_string()
    #2. Read and get cities information from cities_data in SQL
    cities_info = read_cities_data(connection_string)
    #3. Get forecast data in weather_df from the API
    weather_df = weather_cities(cities_info)
    #4. Save forecast weather data in SQL
    save_weather_data(weather_df,connection_string)
    return "Data has been updated"


In [377]:
receive_and_send_data()

'Data has been updated'